# Base firestore

* jump to relevant section after this

In [1]:
import pandas as pd
import firebase_admin
from IPython import display
from IPython.display import display, Markdown,Image

from firebase_admin import firestore
from firebase_admin import credentials

class Firestore:
    def __init__(self,service_account_file="C:/i/auth/run-pix-092258e3cb1b.json"):
        cred = credentials.Certificate(service_account_file)
        try: self.app = firebase_admin.initialize_app(cred)
        except: self.app = firebase_admin.get_app()
        self.fstore=firestore.client()
    "delete all subnodes"
    def delete_all_docs(self,collection):
        for n in collection.get():
            print(n.__dict__)
            n._reference.delete()
    def collection(self):
        return self.fstore
   
runpix=Firestore()
from firebase_admin import storage
bucket=firebase_admin.storage.bucket(name='run-pix.appspot.com', app=None) 



In [2]:
"""
    os Dirctory
"""
import os
from datetime import datetime

def getFileName(file):
    return file.split("\\")[-1]
def getFileIsoDate(file):
    return datetime.fromtimestamp( os.path.getctime(file) ).isoformat()

def dir2df(rootPath):
    arr=[]
    for root, dirs, files in os.walk(rootPath, topdown=False):
        for name in files:
            arr.append({'type':'file', "root":root, 'path':name})
        for name in dirs:
            arr.append({'type':'dir', "root":root, 'path':name})
    return pd.DataFrame(arr)


In [3]:
"""
    firebase Storage
"""
blobs=[blob for blob in bucket.list_blobs()]
df_blob=pd.DataFrame([{"id":b.id,"name":b.name,"blob":b} for b in blobs])


In [4]:
"""
    firestore
"""
def _f(x): 
    _={"id":x.id,'ref':x}
    try: _.update(x.get().to_dict())
    except: pass
    return _
def fs2Df(collPath):
    return pd.DataFrame([_f(_) for _ in runpix.fstore.collection(collPath).list_documents()])

In [5]:
"""
Utilities
"""
#runpix.delete_all_docs(runpix.fstore.collection('races').document('test').collection('readings'))

'\nUtilities\n'

# Races 

In [6]:
races=runpix.fstore.collection('races')
"list all races"

df_races=fs2Df('races')
df_races

,id,ref,Date,bibPatterm,Waypoints,Name,Location,bibPattern,photoStatus
0,default,<google.cloud.firestore_v1.document.DocumentRe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mychoice23apr,<google.cloud.firestore_v1.document.DocumentRe...,YYYY-MM-DD,\d\d\d\d,"[venue, start, end]",Default Race Year Month,DC,NaN,NaN
2,mychoice23feb,<google.cloud.firestore_v1.document.DocumentRe...,2023-02-11,NaN,"[venue, end]",My Choice 2023 February,GT,\d\d\d\d,[available]
3,mychoice23jan,<google.cloud.firestore_v1.document.DocumentRe...,2023-03-12,NaN,"[venue, start, end]",My Choice 2023 Januaryr,DC,NaN,NaN
4,mychoice23mar,<google.cloud.firestore_v1.document.DocumentRe...,2023-03-12,NaN,"[venue, start, end]",My Choice 2023 March,DC,NaN,NaN
5,werun2023,<google.cloud.firestore_v1.document.DocumentRe...,2023-03-05,NaN,"[venue, route]",We Run 2023 (4th Edition),Nigadi Pradhikaram,\d{4},[available]


## list all blobs

In [7]:
df_blob[:3].apply(lambda x: x['name'].split("/").pop(),
                  axis=1)

0    2023-03-13T19:23:42.882584~general~vaibhav~_L3...
1    2023-03-13T19:24:34.262257~general~vaibhav~_L3...
2    2023-03-13T19:25:41.041091~general~vaibhav~_L3...
dtype: object

## other

In [ ]:
"query examples"
{_.id:_.to_dict() for _ in races.where('Location','==','GT').stream()}

In [ ]:
"""
COPY RACE
"""
def copyRace(fromId,toId):
    fromRace=races.document(fromId).get().to_dict()
    print(fromRace)
    return races.document(toId).set(fromRace)

# copyRace('mychoice23apr','mychoice23mar')
# copyRace('mychoice23mar','mychoice23jan')

In [ ]:
def addNewRace(id):
    raceData = {'Date': "YYYY-MM-DD", 
                'Waypoints': ['venue', 'start', 'end'], 
                'Name': 'Default Race Year Month', 
                'Location': 'DC',
                'bibPatterm': r'\d\d\d\d'
               }
    update_time, city_ref = races.document(id).set(raceData)
    print(f'Added document with id {city_ref.id}')
    
addNewRace('mychoice23apr')

# Misc

# Images (Storage)

In [ ]:
images = runpix.fstore.collection('images')
for doc in images.stream():
    print(f"{doc.id} => {doc.to_dict()['textAnnotations']}")

# Storage

In [ ]:
# for i, b in enumerate(blobs):
#     print (i,b.name)
all_blobs_l=df_blob.name.apply(lambda x : x.split("~")[-1])

In [ ]:
"signed URL"
blobs[3].generate_signed_url(pd.Timestamp.today())

In [ ]:

from base64 import b64decode
base64_data = blocs[1].download_as_bytes() #"iVBORw0KGgoAAAANSUhEUgAABL ...  the rest of data "

In [ ]:
base64_data[:100]

In [ ]:
display.Image(b64decode(base64_data))

In [ ]:
# def newname(x):
#     arr=x.split('/')
#     arr[]
# gs://run-pix.appspot.com/thumbs/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg
for i,b in enumerate(blobs):
    if 'thumbs/2023' in b.name :#and b.content_type=='image/jpeg':        
        newName="thumbs/mychoice23feb/"+b.name.split('/')[-1]
        print(i,b.name,newName)
        bucket.copy_blob(b,bucket,newName)

In [ ]:

".generate_signed_url() needs date"
new_id="run-pix.appspot.com/thumbs/mychoice23feb/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/1678882880166980"
blobs[72].content_type

In [ ]:
bucket.copy_blob()

# Bulk Bib Upload

Data copied from clipboard from Excel:

Columns copied are (case sensitive)
* Bib
* Name
* Status
* Race

In [14]:
# raceId = 'werun2023'
display(Markdown(f"## Creating Bib list for {raceId}"))
bibs=runpix.fstore.collection(f'races/{raceId}/bibs')

## Creating Bib list for mychoice23mar

In [15]:
"Check Race details"
def dictUtf(mydict): 
    return {k: str(v).encode("utf-8") for k,v in mydict.items()}

races.document(raceId).get().to_dict()

{'Date': '2023-03-12',
 'photoStatus': ['available'],
 'Waypoints': ['venue', 'start', 'end'],
 'Name': 'My Choice 2023 March',
 'Location': 'DC'}

##  Copy the data
Data copied from clipboard from Excel:

Columns copied are (case sensitive)
* Bib
* Name
* Status
* Race

In [22]:
df_startlist=pd.read_clipboard()
display("columns in clipboard:",df_startlist.columns)
if 'BIB Number' in df_startlist:
    df_startlist=df_startlist.rename(columns={'BIB Number': 'Bib', 'fullname': 'Name',  })
    df_startlist['Status']='From sheet'
    df_startlist['Race']='My Choice'
_required_columns='Bib	Name	Status	Race'.split('\t')
_found_columns = [_x for _x in _required_columns if  _x in df_startlist.columns]
display("columns found ",_found_columns)
if len(_found_columns)<len(_required_columns): print("PLEASE WAIT "+('X'*10+' ')*3)


'columns in clipboard:'

Index(['BIB Number', 'First', 'Last', 'Email', 'fullname', 'RegiId', 'Gender'], dtype='object')

'columns found '

['Bib', 'Name', 'Status', 'Race']

In [23]:
df_startlist=df_startlist[_required_columns]

In [24]:
"code"
bibtoDict=lambda x: x.to_dict()
bibtoDict(df_startlist.loc[0,:])  # test

def saveBibtoRace(bibData,bibsCollection):
    _bibData=bibtoDict(bibData)
    _bibData['Bib']=str(_bibData['Bib'])
    bibsCollection.document(_bibData['Bib']
                           ).set(_bibData)
saveBibtoRace(df_startlist.loc[0,:],bibsCollection=bibs,)  # test    
# df_startlist.apply(, axis=1)


In [25]:
" upload all bibs"
# df_startlist.apply(bibtoDict, axis=1)  #test
df_startlist.apply(lambda x: saveBibtoRace(x,bibs), axis=1) 

0      None
1      None
2      None
3      None
4      None
       ... 
706    None
707    None
708    None
709    None
710    None
Length: 711, dtype: object

In [26]:
bibs.count().get()
# len(list( bibs.list_documents()))

[[<Aggregation alias=field_1, value=711, readtime=2023-03-26 19:29:13.495820+00:00>]]

# Update images to bibs 

Add list of images to the bibs

In [31]:
df_dir.head()

,type,root,path
0,file,D:\umesh\D K D 2023 2\Untitled Export,1P6A5534.jpg
1,file,D:\umesh\D K D 2023 2\Untitled Export,1P6A5535.jpg
2,file,D:\umesh\D K D 2023 2\Untitled Export,1P6A5536.jpg
3,file,D:\umesh\D K D 2023 2\Untitled Export,1P6A5537.jpg
4,file,D:\umesh\D K D 2023 2\Untitled Export,1P6A5538.jpg


# Bulk Upload images list_documents

In [8]:
import yaml
import os
cfg={}
cfg.update(yaml.safe_load(r"""
uploads:
    werun2023:
        #path: D:\We Run 2023\JPEG
        path: D:\We Run 2023\Vaibhav\JPEG
        fileRange: ["","XX"]
        waypoint: venue
        userid: vaibhav
    mychoice23feb:
        path: D:\umesh\D K D  2023 2\Untitled Export
        waypoint: general
        userid: bcoconut
    mychoice23mar:
        path: D:\DKD-Parag\DKD 2023\DCIM\New folder
        waypoint: general
        userid: bcoconut
"""))


raceId='mychoice23mar'
cfg['uploads'][raceId]

{'path': 'D:\\DKD-Parag\\DKD 2023\\DCIM\\New folder',
 'waypoint': 'general',
 'userid': 'bcoconut'}

In [9]:
df_dir=dir2df(cfg['uploads'][raceId]['path'])
# df_dir['type root'.split()].value_counts()                 
print(f"{raceId} uploading from '{cfg['uploads'][raceId]['path']}',df_dir:{df_dir.shape}\n",df_dir.root.value_counts())

mychoice23mar uploading from 'D:\DKD-Parag\DKD 2023\DCIM\New folder',df_dir:(818, 3)
 D:\DKD-Parag\DKD 2023\DCIM\New folder           706
D:\DKD-Parag\DKD 2023\DCIM\New folder\no Bib    112
Name: root, dtype: int64


In [13]:
"""
Manually rename to retrigger the processing...
- You can change waypoint from venue to general
"""
"* Storage: /uploads/race/time~wpt~user~loc~file    # uploaded images"
i=0
folder=f'uploads/{raceId}/'
import concurrent.futures 
def getNewfileName(file,
                   user=cfg['uploads'][raceId]['userid'],
                   place=cfg['uploads'][raceId]['waypoint']):
    return "~".join([getFileIsoDate(file),place,user,getFileName(file)])

def uploadFile(root,name):
    path=os.path.join(root, name)
    new=folder+getNewfileName(path)
    # blob=bucket.blob(folder+name) #used when renaming to diff filenames
    blob=bucket.blob(new)

    if blob.exists():
        print(stats['files'],'blob exists/rewriting',name,new)
        blob.rewrite(blob)
    else:
        threads.append({new:executor.submit(blob.upload_from_filename,path)})
        print(stats['files'],f'uploading {path} to {new}', )
        # blob.upload_from_filename(path)
        

stats={"files":0,'upl':0}
prefix=f'processed/{raceId}'

all_blobs_l=[blob.name.split("~")[-1] for blob in bucket.list_blobs(prefix=prefix)]
executor=concurrent.futures.ThreadPoolExecutor(max_workers=5)
threads=[]

for lot in cfg['uploads'].keys():
    _dir=cfg['uploads'][lot]['path'].lower()
    print("raceId",lot,_dir,f"{len(all_blobs_l)} in {prefix}  ====================")
    # for root, dirs, files in os.walk(cfg['uploads'][lot]['path'], topdown=False):
    for i,d in df_dir.query(f"(type=='file') and (root.str.lower() == @_dir)"
                             ).iterrows():
        stats['files']+=1
        if not d.path in all_blobs_l:
            uploadFile(d.root,d.path)
            stats['upl']+=1
        else:
            print(prefix,d.path ,"found")
            pass
        # if (stats['files']>20): raise  #stopper
    print(f"{lot}: {stats['files']} files")
print(stats)

raceId werun2023 d:\we run 2023\vaibhav\jpeg 5 in processed/mychoice23mar  ====================
werun2023: 0 files
raceId mychoice23feb d:\umesh\d k d  2023 2\untitled export 5 in processed/mychoice23mar  ====================
mychoice23feb: 0 files
raceId mychoice23mar d:\dkd-parag\dkd 2023\dcim\new folder 5 in processed/mychoice23mar  ====================
processed/mychoice23mar DSC_0449.jpg found
2 blob exists/rewriting DSC_0450.jpg uploads/mychoice23mar/2023-03-15T10:32:44.829733~general~bcoconut~DSC_0450.jpg
3 blob exists/rewriting DSC_0451.jpg uploads/mychoice23mar/2023-03-15T10:32:45.092075~general~bcoconut~DSC_0451.jpg
processed/mychoice23mar DSC_0452.jpg found
processed/mychoice23mar DSC_0453.jpg found
processed/mychoice23mar DSC_0454.jpg found
7 blob exists/rewriting DSC_0455.jpg uploads/mychoice23mar/2023-03-15T10:32:46.040282~general~bcoconut~DSC_0455.jpg
8 blob exists/rewriting DSC_0456.jpg uploads/mychoice23mar/2023-03-15T10:32:46.270432~general~bcoconut~DSC_0456.jpg
9 blo

In [ ]:
"""
    find object from gcs
    
"""
display(Markdown("## rename blob"))
# bucket.blob('uploads/werun2023/_L3A3192.jpg').rewrite
# blob.rewrite?
bname='2023-03-13T19:23:14.739819~general~vaibhav~_L3A2997.jpg'
def renameBlob(bname):
    bucket.rename_blob(bucket.blob(bname),folder+bname)
# renameBlob(bname) 
"use 2: existence check"
bucket.blob('error in getDownloadURL thumbs/werun2023/2023-03-13T19:23:42.882584~general~vaibhav~_L3A3007.jpg').exists()

## check storage file (which folder?)

In [ ]:
# check existance
# _path=r"processed/mychoice23feb/2023-02-12T01:28:29.364Z^venue^avinashmane$gmail.com^20230212_065828.jpg"
_path="processed/werun2023/2023-03-13T19:25:41.041091~general~vaibhav~_L3A3047.jpg"
[_,_raceId,_name]=_path.split("/")

print(_,_raceId,_name,)
for r in ['default',_raceId]:
    for t in 'processed thumbs uploads'.split():
        newName="/".join([t,r,_name])
        _blob=bucket.blob(newName)
        print(newName,_blob.exists() )


## check all images in the race

In [ ]:
df_images=fs2Df(f'races/{raceId}/images')
df_images.shape

In [ ]:
# df_images['imagePath'].value_counts()
def checkBlobs(x,types=['processed','thumbs']):
    return [bucket.blob(f'{typ}/{raceId}/{x}').exists() 
                                        for typ in types]
def moveBlob(x,typ='thumbs'): #'processed',
    correctName=f'{typ}/{raceId}/{x}'
    if not bucket.blob(correctName).exists() :
        defaultThumb=bucket.blob(f'{typ}/default/{x}')
        
        if defaultThumb.exists():
            bucket.rename_blob(defaultThumb,f'{typ}/{raceId}/{x}')
            print(f"{correctName} not exists, moving from default")
        else:
            procBlob=bucket.blob(f'uploads/{raceId}/{x}')
            if procBlob.exists():
                print(f"{procBlob.name} rename the blob to retrigger functions")
                procBlob.rewrite(procBlob)
                
            else:
                procBlob.upload_from_filename(path4file+x.split("~")[-1])
                print(f"need to upload {x}")
    else:
        print(f"{correctName} exists")

path4file=r"D:\We Run 2023\JPEG\\"
display(raceId, path4file)
df_images['imagePath'].apply(lambda x: moveBlob(x))

In [ ]:
def deleteFSimages4missingBlobs(x,types=['processed','thumbs']):
    chk = checkBlobs(x.imagePath)
    if not any(chk):
        print(chk,x,x.imagePath)
        x.ref.delete()
df_images.apply(deleteFSimages4missingBlobs ,axis=1)

In [ ]:
df_blob.query("name.str.contains('G017')")

# Other

In [ ]:
bucket.blob('processed/werun2023/2023-03-13T19:25:41.041091~general~vaibhav~_L3A3047.jpg').public_url

#Files to be deleted

raceId mychoice23feb d:\umesh\d k d  2023 2\untitled export 3 in processed/mychoice23feb
1 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5534.jpg to uploads/werun2023/2023-03-16T13:57:47.954854~general~bcoconut~1P6A5534.jpg
2 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5535.jpg to uploads/werun2023/2023-03-16T13:57:48.517336~general~bcoconut~1P6A5535.jpg
3 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5536.jpg to uploads/werun2023/2023-03-16T13:57:48.861077~general~bcoconut~1P6A5536.jpg
4 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5537.jpg to uploads/werun2023/2023-03-16T13:57:49.329810~general~bcoconut~1P6A5537.jpg
5 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5538.jpg to uploads/werun2023/2023-03-16T13:57:49.829797~general~bcoconut~1P6A5538.jpg
6 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5539.jpg to uploads/werun2023/2023-03-16T13:57:50.673521~general~bcoconut~1P6A5539.jpg
7 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5540.jpg to uploads/werun2023/2023-03-16T13:57:51.251623~general~bcoconut~1P6A5540.jpg
8 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5541.jpg to uploads/werun2023/2023-03-16T13:57:52.095346~general~bcoconut~1P6A5541.jpg
9 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5542.jpg to uploads/werun2023/2023-03-16T13:57:52.798449~general~bcoconut~1P6A5542.jpg
10 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5543.jpg to uploads/werun2023/2023-03-16T13:57:53.782791~general~bcoconut~1P6A5543.jpg
11 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5544.jpg to uploads/werun2023/2023-03-16T13:57:54.360898~general~bcoconut~1P6A5544.jpg
12 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5545.jpg to uploads/werun2023/2023-03-16T13:57:54.907755~general~bcoconut~1P6A5545.jpg
13 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5546.jpg to uploads/werun2023/2023-03-16T13:57:55.392115~general~bcoconut~1P6A5546.jpg
14 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5547.jpg to uploads/werun2023/2023-03-16T13:57:55.892099~general~bcoconut~1P6A5547.jpg
15 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5548.jpg to uploads/werun2023/2023-03-16T13:57:56.204588~general~bcoconut~1P6A5548.jpg
16 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5549.jpg to uploads/werun2023/2023-03-16T13:57:56.626450~general~bcoconut~1P6A5549.jpg
17 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5550.jpg to uploads/werun2023/2023-03-16T13:57:57.032689~general~bcoconut~1P6A5550.jpg
18 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5551.jpg to uploads/werun2023/2023-03-16T13:57:57.501423~general~bcoconut~1P6A5551.jpg
19 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5552.jpg to uploads/werun2023/2023-03-16T13:57:57.938908~general~bcoconut~1P6A5552.jpg
20 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5553.jpg to uploads/werun2023/2023-03-16T13:57:58.438893~general~bcoconut~1P6A5553.jpg
21 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5554.jpg to uploads/werun2023/2023-03-16T13:57:59.110745~general~bcoconut~1P6A5554.jpg
22 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5555.jpg to uploads/werun2023/2023-03-16T13:57:59.876345~general~bcoconut~1P6A5555.jpg
23 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5556.jpg to uploads/werun2023/2023-03-16T13:58:00.516953~general~bcoconut~1P6A5556.jpg
24 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5557.jpg to uploads/werun2023/2023-03-16T13:58:00.923188~general~bcoconut~1P6A5557.jpg
25 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5558.jpg to uploads/werun2023/2023-03-16T13:58:01.376301~general~bcoconut~1P6A5558.jpg
26 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5559.jpg to uploads/werun2023/2023-03-16T13:58:01.907535~general~bcoconut~1P6A5559.jpg
27 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5560.jpg to uploads/werun2023/2023-03-16T13:58:02.610633~general~bcoconut~1P6A5560.jpg
28 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5561.jpg to uploads/werun2023/2023-03-16T13:58:03.048121~general~bcoconut~1P6A5561.jpg
29 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5562.jpg to uploads/werun2023/2023-03-16T13:58:03.532478~general~bcoconut~1P6A5562.jpg
30 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5563.jpg to uploads/werun2023/2023-03-16T13:58:03.969964~general~bcoconut~1P6A5563.jpg
31 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5564.jpg to uploads/werun2023/2023-03-16T13:58:04.469950~general~bcoconut~1P6A5564.jpg
32 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5565.jpg to uploads/werun2023/2023-03-16T13:58:05.126179~general~bcoconut~1P6A5565.jpg
33 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5566.jpg to uploads/werun2023/2023-03-16T13:58:05.501164~general~bcoconut~1P6A5566.jpg
34 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5567.jpg to uploads/werun2023/2023-03-16T13:58:05.923026~general~bcoconut~1P6A5567.jpg
35 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5568.jpg to uploads/werun2023/2023-03-16T13:58:06.313642~general~bcoconut~1P6A5568.jpg
36 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5569.jpg to uploads/werun2023/2023-03-16T13:58:06.766748~general~bcoconut~1P6A5569.jpg
37 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5570.jpg to uploads/werun2023/2023-03-16T13:58:07.094863~general~bcoconut~1P6A5570.jpg
38 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5571.jpg to uploads/werun2023/2023-03-16T13:58:07.594847~general~bcoconut~1P6A5571.jpg
39 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5572.jpg to uploads/werun2023/2023-03-16T13:58:07.938586~general~bcoconut~1P6A5572.jpg
40 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5573.jpg to uploads/werun2023/2023-03-16T13:58:08.282328~general~bcoconut~1P6A5573.jpg
41 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5575.jpg to uploads/werun2023/2023-03-16T13:58:08.735436~general~bcoconut~1P6A5575.jpg
42 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5576.jpg to uploads/werun2023/2023-03-16T13:58:09.141680~general~bcoconut~1P6A5576.jpg
43 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5577.jpg to uploads/werun2023/2023-03-16T13:58:09.672905~general~bcoconut~1P6A5577.jpg
44 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5579.jpg to uploads/werun2023/2023-03-16T13:58:10.047894~general~bcoconut~1P6A5579.jpg
45 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5580.jpg to uploads/werun2023/2023-03-16T13:58:10.376008~general~bcoconut~1P6A5580.jpg
46 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5581.jpg to uploads/werun2023/2023-03-16T13:58:10.704123~general~bcoconut~1P6A5581.jpg
47 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5582.jpg to uploads/werun2023/2023-03-16T13:58:11.110360~general~bcoconut~1P6A5582.jpg
48 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5583.jpg to uploads/werun2023/2023-03-16T13:58:11.454099~general~bcoconut~1P6A5583.jpg
49 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5585.jpg to uploads/werun2023/2023-03-16T13:58:11.922834~general~bcoconut~1P6A5585.jpg
50 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5586.jpg to uploads/werun2023/2023-03-16T13:58:12.297822~general~bcoconut~1P6A5586.jpg
51 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5587.jpg to uploads/werun2023/2023-03-16T13:58:12.657186~general~bcoconut~1P6A5587.jpg
52 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5588.jpg to uploads/werun2023/2023-03-16T13:58:12.969674~general~bcoconut~1P6A5588.jpg
53 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5589.jpg to uploads/werun2023/2023-03-16T13:58:13.438409~general~bcoconut~1P6A5589.jpg
54 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5590.jpg to uploads/werun2023/2023-03-16T13:58:13.735275~general~bcoconut~1P6A5590.jpg
55 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5591.jpg to uploads/werun2023/2023-03-16T13:58:14.079020~general~bcoconut~1P6A5591.jpg
56 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5592.jpg to uploads/werun2023/2023-03-16T13:58:14.407129~general~bcoconut~1P6A5592.jpg
57 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5594.jpg to uploads/werun2023/2023-03-16T13:58:14.703994~general~bcoconut~1P6A5594.jpg
58 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5595.jpg to uploads/werun2023/2023-03-16T13:58:15.000858~general~bcoconut~1P6A5595.jpg
59 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5596.jpg to uploads/werun2023/2023-03-16T13:58:15.391475~general~bcoconut~1P6A5596.jpg
60 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5597.jpg to uploads/werun2023/2023-03-16T13:58:15.813334~general~bcoconut~1P6A5597.jpg
61 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5598.jpg to uploads/werun2023/2023-03-16T13:58:16.360190~general~bcoconut~1P6A5598.jpg
62 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5599.jpg to uploads/werun2023/2023-03-16T13:58:16.703928~general~bcoconut~1P6A5599.jpg
63 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5600.jpg to uploads/werun2023/2023-03-16T13:58:16.953921~general~bcoconut~1P6A5600.jpg
64 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5601.jpg to uploads/werun2023/2023-03-16T13:58:17.250787~general~bcoconut~1P6A5601.jpg
65 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5602.jpg to uploads/werun2023/2023-03-16T13:58:17.563277~general~bcoconut~1P6A5602.jpg
66 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5603.jpg to uploads/werun2023/2023-03-16T13:58:17.844517~general~bcoconut~1P6A5603.jpg
67 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5604.jpg to uploads/werun2023/2023-03-16T13:58:18.219508~general~bcoconut~1P6A5604.jpg
68 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5605.jpg to uploads/werun2023/2023-03-16T13:58:18.516370~general~bcoconut~1P6A5605.jpg